# Proportion Tests

# 1. One-sample proportion tests

<b>1.1 t for proportions?</b>

Some of the hypothesis tests in this course have used a  test statistic <i>z</i> and some have used a <i>t</i> test statistic. To get the correct p-value, you need to use the right type of test statistic.

Do tests of proportion(s) use a  or a  test statistic and why?

- <i>t</i> : There are two estimates used for unknown values in the test statistic for proportion(s).

- <i>z</i> : Since the population standard deviation is always known for proportions, we always compute z-scores.

- <b><font color='green'><i>z</i> : The test statistic for proportion(s) has only one estimate of a parameter instead of two.</font></b>

- <i>t</i> : Proportions are ratios, so you need to estimate the numerator and the denominator.

The t-test is needed for tests of mean(s) since you are estimating two unknown quantities, which leads to more variability.

<b>1.2 Test for single proportions</b>

In Chapter 1, you calculated a p-value for a test hypothesizing that the proportion of late shipments was greater than 6%. In that chapter, you used a bootstrap distribution to estimate the standard error of the statistic. An alternative is to use an equation for the standard error based on the sample proportion, hypothesized proportion, and sample size.

$$
z = \frac{\hat{p}-p_{o}}{\sqrt{\frac{p_{o}*(1-p_{o})}{n}}}
$$

In [3]:
# Import the late_shipment dataframe
import pandas as pd
import numpy as np

late_shipments = pd.read_feather("C:\\Users\\yazan\\Desktop\\Data_Analytics\\9-Introduction to Hypothesis Testing\\Datasets\\late_shipments.feather")


In [4]:
from scipy.stats import norm

# Hypothesize that the proportion of late shipments is 6%
p_0 = 0.06

# Calculate the sample proportion of late shipments
p_hat = (late_shipments['late'] == "Yes").mean()

# Calculate the sample size
n = len(late_shipments)

# Calculate the numerator and denominator of the test statistic
numerator = p_hat - p_0
denominator = np.sqrt(p_0 * (1 - p_0) / n)

# Calculate the test statistic
z_score = numerator / denominator

# Calculate the p-value from the z-score
p_value = 1 - norm.cdf(z_score)

# Print the p-value
print(p_value)

0.44703503936503364


While bootstrapping can be used to estimate the standard error of any statistic, it is computationally intensive. For proportions, using a simple equation of the hypothesized proportion and sample size is easier to compute.